# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,23.59,86,79,7.73,PN,1714164376
1,1,saskatoon,52.1168,-106.6345,12.34,38,94,5.92,CA,1714164290
2,2,kharp,66.8014,65.8081,-8.57,97,82,1.14,RU,1714164377
3,3,kotovsk,47.7531,29.5320,7.51,87,67,3.04,UA,1714164377
4,4,broome,42.2506,-75.8330,14.82,32,11,0.97,US,1714164377


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = city_data_df['Humidity'],
    color = city_data_df['City'],
    alpha= 0.5    
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (_color,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= 20) & 
                                (city_data_df['Humidity'] <= 100) & 
                                (city_data_df['Wind Speed'] < 5) &
                                (city_data_df['Cloudiness'] == 0)
] 
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
133,133,tura,25.5198,90.2201,24.51,68,0,3.64,IN,1714164408
190,190,ratnagiri,16.9833,73.3000,27.13,83,0,3.84,IN,1714164422
237,237,nueva gerona,21.8833,-82.8000,30.21,35,0,4.65,CU,1714164434
238,238,reguiba,33.5639,6.7033,23.26,34,0,2.39,DZ,1714164434
262,262,sur,22.5667,59.5289,28.13,64,0,2.75,OM,1714164440
283,283,awjilah,29.1081,21.2869,21.09,15,0,3.73,LY,1714164445
337,337,recani,31.2846,-4.2688,23.38,20,0,4.00,MA,1714164458
343,343,xam nua,20.4197,104.0439,21.11,54,0,1.71,LA,1714164459
345,345,sharjah,25.3573,55.4033,27.06,56,0,2.43,AE,1714164403
423,423,sabancuy,18.9667,-91.1833,35.30,40,0,3.33,MX,1714164479


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_to_copy = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = ideal_weather_df[columns_to_copy].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
133,tura,IN,25.5198,90.2201,68,
190,ratnagiri,IN,16.9833,73.3000,83,
237,nueva gerona,CU,21.8833,-82.8000,35,
238,reguiba,DZ,33.5639,6.7033,34,
262,sur,OM,22.5667,59.5289,64,
283,awjilah,LY,29.1081,21.2869,15,
337,recani,MA,31.2846,-4.2688,20,
343,xam nua,LA,20.4197,104.0439,54,
345,sharjah,AE,25.3573,55.4033,56,
423,sabancuy,MX,18.9667,-91.1833,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    coordinates = f"{row['Lng']},{row['Lat']}"
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{coordinates},{radius}"
    params["bias"] = f"proximity:{coordinates}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: Hotel Polo Orchid
ratnagiri - nearest hotel: Vihar
nueva gerona - nearest hotel: Hotel Bamboo
reguiba - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
awjilah - nearest hotel: No hotel found
recani - nearest hotel: Hotel Merzouga
xam nua - nearest hotel: Phetmeuangsam Hotel
sharjah - nearest hotel: shakeel bhai room
sabancuy - nearest hotel: Hotel plaza sabancuy
dondo - nearest hotel: No hotel found
ad dabbah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
133,tura,IN,25.5198,90.2201,68,Hotel Polo Orchid
190,ratnagiri,IN,16.9833,73.3000,83,Vihar
237,nueva gerona,CU,21.8833,-82.8000,35,Hotel Bamboo
238,reguiba,DZ,33.5639,6.7033,34,No hotel found
262,sur,OM,22.5667,59.5289,64,Sur Hotel
283,awjilah,LY,29.1081,21.2869,15,No hotel found
337,recani,MA,31.2846,-4.2688,20,Hotel Merzouga
343,xam nua,LA,20.4197,104.0439,54,Phetmeuangsam Hotel
345,sharjah,AE,25.3573,55.4033,56,shakeel bhai room
423,sabancuy,MX,18.9667,-91.1833,40,Hotel plaza sabancuy


In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = 'City',
    size = 'Humidity', 
    hover_cols = ['Humidity', 'Hotel Name', 'Country'],
    alpha= 0.5    
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)